In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#get libraries.
import numpy as np
import pandas as pd
print('Done')

Done


In [ ]:
data = pd.DataFrame(pd.read_excel("/content/gdrive/MyDrive/Links_Classification_Task.xlsx"))

In [ ]:
print(data.columns)

Index(['link_id', 'page_source_path', 'link_url', 'link_domain_name',
       'content_name', 'trans_content_name', 'alt_content_names',
       'campaign_name', 'number_of_episodes', 'class'],
      dtype='object')


In [ ]:
#Let's keep only the columns that we're going to use
#data = data[['link_url','class']]
data.head()

,link_id,page_source_path,link_url,link_domain_name,content_name,trans_content_name,alt_content_names,campaign_name,number_of_episodes,class
0,2554338,doms/a068feac8d33e1d11379987dfef4beefd0a6e1ef....,https://video.aflamyeg.com/watch.php?vid=9d92e...,video.aflamyeg.com,البرنس,El Prince,"Al Prince,El Prince",RH_Campaign_البرنس,30,related
1,2746768,doms/030946acbb0c3257000915fa36f98b34cc1ed349....,https://b.brstej.com/watch.php?vid=a3fae3d8b,b.brstej.com,قدرى و نصيبى,NaN,Dilli Wali Thakur Gurls,Qadari w Nasibi -قدرى ونصيبي,22,related
2,2439834,doms/e1cbb7a8f40068d13bf16392618497138318e14b....,https://www.alfantime.com/video/watch.php?vid=...,alfantime.com,The Handmaid’s Tale,NaN,"حكاية أمة,قصة الخادمة",The Handmaid’s Tale,46,related
3,2554434,doms/26a072b1a330967cbae92c49c343b3a91f1ecf91....,https://www.fou2sh.net/video/3d48f74d0,fou2sh.net,البرنس,El Prince,"Al Prince,El Prince",RH_Campaign_البرنس,30,related
4,2685117,doms/b04bdd3cbd554d838cbe63b94ecddefff170399e....,http://idxaljku.gkspiter.ru/21.html,idxaljku.gkspiter.ru,من أنا,NaN,من انا الهندى,من أنا,30,unrelated


In [ ]:
#check empty data
data.isnull().sum()

link_id                  0
page_source_path         0
link_url                 0
link_domain_name         0
content_name             0
trans_content_name    5085
alt_content_names      851
campaign_name            0
number_of_episodes       0
class                   10
dtype: int64

In [ ]:
len(data)

8000

In [ ]:
#drop trans_content_name as 5080 of 8000 of the data is nan so it will has bad effect on the dataset as it is condidered as noise 
data.drop('trans_content_name',axis='columns', inplace=True)

In [ ]:
data.isnull().sum()

link_id                 0
page_source_path        0
link_url                0
link_domain_name        0
content_name            0
alt_content_names     851
campaign_name           0
number_of_episodes      0
class                  10
dtype: int64

In [ ]:
#drop rows contain nan
data.dropna(subset=['alt_content_names', 'class'], inplace=True)

In [ ]:
data.isnull().sum()
#len(data)

link_id               0
page_source_path      0
link_url              0
link_domain_name      0
content_name          0
alt_content_names     0
campaign_name         0
number_of_episodes    0
class                 0
dtype: int64

In [ ]:
len(data)

7140

In [ ]:
data.head()

,link_id,page_source_path,link_url,link_domain_name,content_name,alt_content_names,campaign_name,number_of_episodes,class
0,2554338,doms/a068feac8d33e1d11379987dfef4beefd0a6e1ef....,https://video.aflamyeg.com/watch.php?vid=9d92e...,video.aflamyeg.com,البرنس,"Al Prince,El Prince",RH_Campaign_البرنس,30,related
1,2746768,doms/030946acbb0c3257000915fa36f98b34cc1ed349....,https://b.brstej.com/watch.php?vid=a3fae3d8b,b.brstej.com,قدرى و نصيبى,Dilli Wali Thakur Gurls,Qadari w Nasibi -قدرى ونصيبي,22,related
2,2439834,doms/e1cbb7a8f40068d13bf16392618497138318e14b....,https://www.alfantime.com/video/watch.php?vid=...,alfantime.com,The Handmaid’s Tale,"حكاية أمة,قصة الخادمة",The Handmaid’s Tale,46,related
3,2554434,doms/26a072b1a330967cbae92c49c343b3a91f1ecf91....,https://www.fou2sh.net/video/3d48f74d0,fou2sh.net,البرنس,"Al Prince,El Prince",RH_Campaign_البرنس,30,related
4,2685117,doms/b04bdd3cbd554d838cbe63b94ecddefff170399e....,http://idxaljku.gkspiter.ru/21.html,idxaljku.gkspiter.ru,من أنا,من انا الهندى,من أنا,30,unrelated


In [ ]:
#covert class to 1 for related , 0 unrelated
data = data.replace({'class': {'related': 1, 
                                'unrelated':0}})

In [ ]:
data.head()

,link_id,page_source_path,link_url,link_domain_name,content_name,alt_content_names,campaign_name,number_of_episodes,class
0,2554338,doms/a068feac8d33e1d11379987dfef4beefd0a6e1ef....,https://video.aflamyeg.com/watch.php?vid=9d92e...,video.aflamyeg.com,البرنس,"Al Prince,El Prince",RH_Campaign_البرنس,30,1
1,2746768,doms/030946acbb0c3257000915fa36f98b34cc1ed349....,https://b.brstej.com/watch.php?vid=a3fae3d8b,b.brstej.com,قدرى و نصيبى,Dilli Wali Thakur Gurls,Qadari w Nasibi -قدرى ونصيبي,22,1
2,2439834,doms/e1cbb7a8f40068d13bf16392618497138318e14b....,https://www.alfantime.com/video/watch.php?vid=...,alfantime.com,The Handmaid’s Tale,"حكاية أمة,قصة الخادمة",The Handmaid’s Tale,46,1
3,2554434,doms/26a072b1a330967cbae92c49c343b3a91f1ecf91....,https://www.fou2sh.net/video/3d48f74d0,fou2sh.net,البرنس,"Al Prince,El Prince",RH_Campaign_البرنس,30,1
4,2685117,doms/b04bdd3cbd554d838cbe63b94ecddefff170399e....,http://idxaljku.gkspiter.ru/21.html,idxaljku.gkspiter.ru,من أنا,من انا الهندى,من أنا,30,0


In [ ]:
#get url data alone 
urldata =data.filter(['link_url'],axis=1)


In [ ]:
urldata.head()

,link_url
0,https://video.aflamyeg.com/watch.php?vid=9d92e...
1,https://b.brstej.com/watch.php?vid=a3fae3d8b
2,https://www.alfantime.com/video/watch.php?vid=...
3,https://www.fou2sh.net/video/3d48f74d0
4,http://idxaljku.gkspiter.ru/21.html


In [ ]:
#save urldata at csv file
urldata.to_csv("urldata.csv",index=False)
#/content/gdrive/MyDrive/Digisay_task

In [ ]:
page_source_path_data =data.filter(['page_source_path'],axis=1)

In [ ]:
page_source_path_data.head()

,page_source_path
0,doms/a068feac8d33e1d11379987dfef4beefd0a6e1ef....
1,doms/030946acbb0c3257000915fa36f98b34cc1ed349....
2,doms/e1cbb7a8f40068d13bf16392618497138318e14b....
3,doms/26a072b1a330967cbae92c49c343b3a91f1ecf91....
4,doms/b04bdd3cbd554d838cbe63b94ecddefff170399e....


In [ ]:
page_source_path_data.to_csv("page_source_path_data.csv",index=False)

In [ ]:
#extract feature from url data

In [ ]:
!pip install python-whois

     |████████████████████████████████| 91 kB 10.8 MB/s 
  Created wheel for python-whois: filename=python_whois-0.7.3-py3-none-any.whl size=87721 sha256=bb1c245feeeb5a7f10a364a758f10da1464a877f521e4644428b828f557c029d
  Stored in directory: /root/.cache/pip/wheels/11/05/f7/895ce5a73665f77c8274a7d55e34fb3e6b4abbb9a7637e215b
Successfully built python-whois


In [ ]:
!pip install tldextract

     |████████████████████████████████| 87 kB 6.7 MB/s 


In [ ]:
import whois
import datetime
from os.path import splitext
import tldextract
from urllib.parse import urlparse

In [ ]:
#function to perform whois on given url
def perform_whois(url):
    try:
        whois_result = whois.whois(url)
        return whois_result
    except Exception:
        return False

In [ ]:
#function to fetch the website age in days using URL created_date
def get_registered_date_in_days(whois_result):
    if(whois_result!=False):
        created_date = whois_result.creation_date
        if((created_date is not None) and (type(created_date)!=str)):
            if(type(created_date)==list):
                created_date=created_date[0]
            today_date=datetime.datetime.now()
            days = (today_date-created_date).days
            return days
        else:
            return -1
    else:
        return -1

In [ ]:
#function to fetch the website expiry date in days using URL expiration_date
def get_expiration_date_in_days(whois_result):
    if(whois_result!=False):
        expiration_date = whois_result.expiration_date
        if((expiration_date is not None) and (type(expiration_date)!=str)):
            if(type(expiration_date)==list):
                expiration_date = expiration_date[0]
            today_date=datetime.datetime.now()
            days = (expiration_date-today_date).days
            return days
        else:
            return -1
    else:
        return -1

In [ ]:
#function to fetch the website's last updated date in days using URL updated_date
def get_updated_date_in_days(whois_result):
    if(whois_result!=False):
        updated_date = whois_result.updated_date
        if((updated_date is not None) and (type(updated_date)!=str)):
            if(type(updated_date)==list):
                updated_date = updated_date[0]
            today_date=datetime.datetime.now()
            days = (today_date-updated_date).days
            return days
        else:
            return -1
    else:
        return -1

In [ ]:
def get_dot_count(url):
    return url.count('.')

In [ ]:
def get_url_length(url):
    return len(url)

In [ ]:
def get_digit_count(url):
    return sum(c.isdigit() for c in url)

In [ ]:

def get_special_char_count(url):
    count = 0
    special_characters = [';','+=','_','?','=','&','[',']']
    for each_letter in url:
        if each_letter in special_characters:
            count = count + 1
    return count

In [ ]:
def get_hyphen_count(url):
    return url.count('-')

In [ ]:
def get_double_slash(url):
    return url.count('//')

In [ ]:
def get_single_slash(url):
    return url.count('/')

In [ ]:
def get_at_the_rate(url):
    return url.count('@')

In [ ]:
def get_protocol(url):
    protocol = urlparse(url)
    if(protocol.scheme == 'http'):
        return 1
    else:
        return 0

In [ ]:
def get_protocol_count(url):
    http_count = url.count('http')
    https_count = url.count('https')
    http_count = http_count - https_count #correcting the miscount of https as http
    return (http_count + https_count)

In [ ]:
registered_date_in_days = []
expiration_date_in_days = []
updated_date_in_days = []
dotCount = []
urlLength = []
digitCount = []
specialCharCount = []
hyphenCount = []
doubleSlashCount = []
singleSlashCount = []
atTheRateCount = []
protocol = []
protocolCount = []

In [ ]:
urldata = pd.read_csv('/content/gdrive/MyDrive/urldata.csv')

In [ ]:
urldata.head()

,link_url
0,https://video.aflamyeg.com/watch.php?vid=9d92e...
1,https://b.brstej.com/watch.php?vid=a3fae3d8b
2,https://www.alfantime.com/video/watch.php?vid=...
3,https://www.fou2sh.net/video/3d48f74d0
4,http://idxaljku.gkspiter.ru/21.html


In [ ]:
def extract_all_features():
    counter = 0
    for url in urldata['link_url']:
        counter = counter + 1
        print(counter)
        whois_result = perform_whois(url)
        #Extracting whois features from URLs
        registered_date_in_days.append(get_registered_date_in_days(whois_result))
        expiration_date_in_days.append(get_expiration_date_in_days(whois_result))
        updated_date_in_days.append(get_updated_date_in_days(whois_result))
        #Extracting lexical features from URLs
        dotCount.append(get_dot_count(url))
        urlLength.append(get_url_length(url))
        digitCount.append(get_digit_count(url))
        specialCharCount.append(get_special_char_count(url))
        hyphenCount.append(get_hyphen_count(url))
        doubleSlashCount.append(get_double_slash(url))
        singleSlashCount.append(get_single_slash(url))
        atTheRateCount.append(get_at_the_rate(url))
        protocol.append(get_protocol(url))
        protocolCount.append(get_protocol_count(url))

In [ ]:
extract_all_features()

In [ ]:
print(f'Registered Date list length               : {len(registered_date_in_days)}')
print(f'Expiration Date list length               : {len(expiration_date_in_days)}')
print(f'Updation Date list length                 : {len(updated_date_in_days)}')
print(f'Dot Count list length                     : {len(dotCount)}')
print(f'URL Length list length                    : {len(urlLength)}')
print(f'Digit Count list length                   : {len(digitCount)}')
print(f'Special Character Count list length       : {len(specialCharCount)}')
print(f'Hyphen Count list length                  : {len(hyphenCount)}')
print(f'Double Slash Count list length            : {len(doubleSlashCount)}')
print(f'Single Slash Count list length            : {len(singleSlashCount)}')
print(f'At the Rate(@) Count list length          : {len(atTheRateCount)}')
print(f'ProtocolName Count list length            : {len(protocol)}')
print(f'Protocol Count list length                : {len(protocolCount)}')

Registered Date list length               : 7140
Expiration Date list length               : 7140
Updation Date list length                 : 7140
Dot Count list length                     : 7140
URL Length list length                    : 7140
Digit Count list length                   : 7140
Special Character Count list length       : 7140
Hyphen Count list length                  : 7140
Double Slash Count list length            : 7140
Single Slash Count list length            : 7140
At the Rate(@) Count list length          : 7140
ProtocolName Count list length            : 7140
Protocol Count list length                : 7140


In [ ]:
features_df = pd.DataFrame()
features_df['whois_regDate'] = registered_date_in_days
features_df['whois_expDate'] = expiration_date_in_days
features_df['whois_updatedDate'] = updated_date_in_days
features_df["dot_count"] = dotCount
features_df["url_len"] = urlLength
features_df["digit_count"] = digitCount
features_df["special_count"] = specialCharCount
features_df["hyphen_count"] = hyphenCount
features_df["double_slash"] = doubleSlashCount
features_df["single_slash"] = singleSlashCount
features_df["at_the_rate"] = atTheRateCount
features_df["protocol"] = protocol
features_df["protocol_count"] = protocolCount

In [ ]:
features_df.head()

,whois_regDate,whois_expDate,whois_updatedDate,dot_count,url_len,digit_count,special_count,hyphen_count,double_slash,single_slash,at_the_rate,protocol,protocol_count
0,723,7,314,3,50,7,2,0,1,3,0,0,1
1,1162,298,97,3,44,3,2,0,1,3,0,0,1
2,1268,192,58,3,55,7,2,0,1,4,0,0,1
3,-1,-1,-1,2,38,7,0,0,1,4,0,0,1
4,-1,-1,-1,3,35,2,0,0,1,3,0,1,1


In [ ]:
features_df.shape

(7140, 13)

In [ ]:
features_df.to_csv("features_url.csv",index=False)